In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

# SLIM Model

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

In [4]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)

SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) items with no interactions.


In [5]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13638 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13638 ( 0.1%) Users that have less than 1 test interactions


In [6]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(250, 500),
    "l1_ratio": Real(1e-06, 1e-03),
    "alpha": Real(0.5, 0.9)
}

In [7]:
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = MultiThreadSLIM_SLIMElasticNetRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

In [8]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [9]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [10]:
import os

output_folder_path = "result_experiments/2_SLIM"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [11]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 383, 'l1_ratio': 6.724088573705469e-05, 'alpha': 0.709691630907596}
SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) items with no interactions.


100%|████████████████████████████████████▉| 18056/18059 [08:50<00:00, 65.64it/s]

EvaluatorHoldout: Processed 13638 (100.0%) in 16.84 sec. Users per second: 810
SearchBayesianSkopt: New best config found. Config 0: {'topK': 383, 'l1_ratio': 6.724088573705469e-05, 'alpha': 0.709691630907596} - results: PRECISION: 0.2333333, PRECISION_RECALL_MIN_DEN: 0.2355526, RECALL: 0.0533430, MAP: 0.1144214, MAP_MIN_DEN: 0.1153558, MRR: 0.4636334, NDCG: 0.2422821, F1: 0.0868345, HIT_RATE: 0.9087843, ARHR_ALL_HITS: 0.7264659, NOVELTY: 0.0052812, AVERAGE_POPULARITY: 0.6732088, DIVERSITY_MEAN_INTER_LIST: 0.7879520, DIVERSITY_HERFINDAHL: 0.9787894, COVERAGE_ITEM: 0.0382635, COVERAGE_ITEM_CORRECT: 0.0205438, COVERAGE_USER: 0.9991209, COVERAGE_USER_CORRECT: 0.9079853, DIVERSITY_GINI: 0.0035901, SHANNON_ENTROPY: 6.3155930, RATIO_DIVERSITY_HERFINDAHL: 0.9791670, RATIO_DIVERSITY_GINI: 0.0144876, RATIO_SHANNON_ENTROPY: 0.5097427, RATIO_AVERAGE_POPULARITY: 3.3142354, RATIO_NOVELTY: 0.0310612, 

EvaluatorHoldout: Processed 13638 (100.0%) in 16.78 sec. Users per second: 813
SearchBayesianSkopt


100%|████████████████████████████████████▉| 18056/18059 [09:43<00:00, 30.94it/s]

 19%|███████▏                              | 3400/18059 [01:11<04:55, 49.59it/s]


 39%|██████████████▋                       | 6984/18059 [02:24<03:24, 54.06it/s]


...

100%|████████████████████████████████████▉| 18056/18059 [14:02<00:00, 21.44it/s]

 95%|███████████████████████████████████  | 17096/18059 [06:23<00:23, 40.70it/s]


100%|████████████████████████████████████▉| 18032/18059 [06:44<00:00, 68.91it/s]

EvaluatorHoldout: Processed 13638 (100.0%) in 15.66 sec. Users per second: 871
SearchBayesianSkopt: Config 49 is suboptimal. Config: {'topK': 307, 'l1_ratio': 0.001, 'alpha': 0.5} - results: PRECISION: 0.2344845, PRECISION_RECALL_MIN_DEN: 0.2366011, RECALL: 0.0535587, MAP: 0.1146364, MAP_MIN_DEN: 0.1155953, MRR: 0.4628019, NDCG: 0.2428974, F1: 0.0872000, HIT_RATE: 0.9084910, ARHR_ALL_HITS: 0.7267320, NOVELTY: 0.0053046, AVERAGE_POPULARITY: 0.6592710, DIVERSITY_MEAN_INTER_LIST: 0.8064383, DIVERSITY_HERFINDAHL: 0.9806379, COVERAGE_ITEM: 0.0411983, COVERAGE_ITEM_CORRECT: 0.0223157, COVERAGE_USER: 0.9991209, COVERAGE_USER_CORRECT: 0.9076923, DIVERSITY_GINI: 0.0040919, SHANNON_ENTROPY: 6.4950475, RATIO_DIVERSITY_HERFINDAHL: 0.9810162, RATIO_DIVERSITY_GINI: 0.0165125, RATIO_SHANNON_ENTROPY: 0.5242267, RATIO_AVERAGE_POPULARITY: 3.2456191, RATIO_NOVELTY: 0.0311990, 

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 421.2186
Function value obtained: -0.1146
Curren

100%|████████████████████████████████████▉| 18056/18059 [09:28<00:00, 31.78it/s]


EvaluatorHoldout: Processed 13638 (100.0%) in 18.92 sec. Users per second: 721
SearchBayesianSkopt: Best config evaluated with evaluator_test with constructor data for final test. Config: {'topK': 317, 'l1_ratio': 0.0007473789079929813, 'alpha': 0.5} - results:
CUTOFF: 10 - PRECISION: 0.3891333, PRECISION_RECALL_MIN_DEN: 0.3906828, RECALL: 0.0686281, MAP: 0.2409020, MAP_MIN_DEN: 0.2415824, MRR: 0.6415620, NDCG: 0.4045243, F1: 0.1166786, HIT_RATE: 0.9677372, ARHR_ALL_HITS: 1.2208103, NOVELTY: 0.0053605, AVERAGE_POPULARITY: 0.6226540, DIVERSITY_MEAN_INTER_LIST: 0.8568129, DIVERSITY_HERFINDAHL: 0.9856750, COVERAGE_ITEM: 0.0441331, COVERAGE_ITEM_CORRECT: 0.0307880, COVERAGE_USER: 0.9991209, COVERAGE_USER_CORRECT: 0.9668864, DIVERSITY_GINI: 0.0054042, SHANNON_ENTROPY: 6.9215352, RATIO_DIVERSITY_HERFINDAHL: 0.9860553, RATIO_DIVERSITY_GINI: 0.0217884, RATIO_SHANNON_ENTROPY: 0.5586122, RATIO_AVERAGE_POPULARITY: 3.0693360, RATIO_NOVELTY: 0.0252204, 


SearchBayesianSkopt: Saving model in result

In [12]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['exception_list', 'result_on_test_df', 'result_on_last', 'time_on_train_total', 'hyperparameters_df', 'time_df', 'algorithm_name_recommender', 'metric_to_optimize', 'algorithm_name_search', 'hyperparameters_best_index', 'cutoff_to_optimize', 'time_on_validation_total', 'result_on_validation_best', 'hyperparameters_best', 'time_on_last_df', 'time_on_validation_avg', 'time_on_test_avg', 'time_on_test_total', 'result_on_test_best', 'result_on_validation_df', 'time_on_train_avg'])

In [16]:
hyperparameters_df = search_metadata["hyperparameters_best"]
hyperparameters_df

{'topK': 317, 'l1_ratio': 0.0007473789079929813, 'alpha': 0.5}

In [14]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.233333,0.235553,0.053343,0.114421,0.115356,0.463633,0.242282,0.086834,0.908784,0.726466,...,0.020544,0.999121,0.907985,0.00359,6.315593,0.979167,0.014488,0.509743,3.314235,0.031061
1,10,0.233018,0.235114,0.053103,0.114099,0.115014,0.463553,0.241859,0.086494,0.908638,0.725231,...,0.020101,0.999121,0.907839,0.003513,6.293517,0.979083,0.014177,0.507961,3.3291,0.031021
2,10,0.234389,0.236527,0.05349,0.114698,0.115657,0.463336,0.242901,0.087103,0.909298,0.727019,...,0.021928,0.999121,0.908498,0.004022,6.468245,0.980685,0.016232,0.522064,3.252152,0.031191
3,10,0.232651,0.234735,0.052994,0.113989,0.114897,0.463579,0.241626,0.086325,0.908271,0.724925,...,0.019602,0.999121,0.907473,0.003419,6.25747,0.978728,0.013799,0.505051,3.343824,0.03099
4,10,0.23271,0.23492,0.053171,0.114048,0.114972,0.463659,0.241732,0.086563,0.908564,0.725158,...,0.019713,0.999121,0.907766,0.003446,6.263185,0.978668,0.013907,0.505513,3.336439,0.031012
5,10,0.234235,0.236369,0.053488,0.114708,0.115634,0.464308,0.242954,0.087089,0.910031,0.72783,...,0.022205,0.999121,0.909231,0.00404,6.483792,0.981044,0.016305,0.523318,3.256962,0.03117
6,10,0.232659,0.234762,0.053044,0.114074,0.114987,0.463638,0.241718,0.086392,0.907831,0.725339,...,0.01999,0.999121,0.907033,0.00351,6.289873,0.979002,0.014163,0.507667,3.327967,0.031027
7,10,0.233825,0.23595,0.05342,0.114505,0.115441,0.464095,0.242636,0.086971,0.908124,0.727216,...,0.021873,0.999121,0.907326,0.003975,6.454215,0.980594,0.01604,0.520931,3.258646,0.031174
8,10,0.232043,0.234273,0.053002,0.113683,0.114608,0.4616,0.240985,0.086293,0.906878,0.722561,...,0.019104,0.999121,0.906081,0.003349,6.221072,0.978147,0.013515,0.502114,3.347735,0.030993


In [17]:
recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all.tocsr())
recommender.fit(l1_ratio=hyperparameters_df['l1_ratio'], alpha = hyperparameters_df['alpha'], positive_only=True, topK = hyperparameters_df['topK'])

100%|████████████████████████████████████▉| 18032/18059 [12:01<00:00, 32.71it/s]

In [ ]:
import pandas as pd
K=10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
								sep=",",
								dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, K)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
# submission

from datetime import datetime
now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)

0
3899 5433 16365 16359 8635 640 6061 7143 9130 10586
1
3473 940 14400 16365 16012 14358 640 15119 2665 10118
2
6177 1551 16365 15119 17330 16846 2665 17022 14181 5925
